In [151]:
from inltk.inltk import tokenize

In [152]:
import re
import numpy as np
from nltk import sent_tokenize, word_tokenize
from nltk.cluster.util import cosine_distance

In [153]:
MULTIPLE_WHITESPACE_PATTERN = re.compile(r"\s+", re.UNICODE)

In [154]:
# text_str = '''
#     Those Who Are Resilient Stay In The Game Longer *
#     “On the mountains of truth you can never climb in vain: either you will reach a point higher up today, or you will be training your powers so that you will be able to climb higher tomorrow.” — Friedrich Nietzsche
#     Challenges and setbacks are not meant to defeat you, but promote you. However, I realise after many years of defeats, it can crush your spirit and it is easier to give up than risk further setbacks and disappointments. Have you experienced this before? To be honest, I don’t have the answers. I can’t tell you what the right course of action is; only you will know. However, it’s important not to be discouraged by failure when pursuing a goal or a dream, since failure itself means different things to different people. To a person with a Fixed Mindset failure is a blow to their self-esteem, yet to a person with a Growth Mindset, it’s an opportunity to improve and find new ways to overcome their obstacles. Same failure, yet different responses. Who is right and who is wrong? Neither. Each person has a different mindset that decides their outcome. Those who are resilient stay in the game longer and draw on their inner means to succeed.
# #     '''

text_str = '''
भारतीय टीम को ऑस्ट्रेलिया के खिलाफ 4 मैचों की टेस्ट सीरीज का तीसरा मुकाबला 1 मार्च से इंदौर में खेलना है. सीरीज में इस वक्त भारतीय टीम 2-0 से आगे है. शुरुआती दोनों टेस्ट में भारतीय स्पिनर गेंदबाजों के आगे ऑस्ट्रेलियाई टीम घुटने टेकती नजर आई. मगर मिडिल ऑर्डर बल्लेबाज पीटर हैंड्सकॉम्ब ने दिल्ली टेस्ट में 72 रनों की नाबाद पारी खेली थी.पीटर हैंड्सकॉम्ब ने पूर्व भारतीय उपकप्तान अजिंक्य रहाणे के साथ उस बातचीत को याद किया जिससे उन्हें उपमहाद्वीप में स्पिन गेंदबाजी का सामना करने में मदद की. हैंड्सकॉम्ब ने 2016 में IPL के दौरान रहाणे के साथ ड्रेसिंग रूम शेयर किया था. रहाणे ने इस दौरान हैंड्सकॉम्ब को स्पिनरों को बल्लेबाजी के दौरान पैर और कलाई के उपयोग करने के बारे में बताया था.सिडनी मॉर्निंग हेराल्ड से बात करते हुए पीटर ने कहा, 'वह अपने पिछले पैर का इस्तेमाल करते हुए स्पिनरों को आसानी से मिडविकेट की ओर खेल रहे थे. इससे मैं आश्चर्यचकित हो रहा था. स्पिनरों के खिलाफ रहाणे की बल्लेबाजी देख कर मैं सोच रहा था कि मुझे यह सिखाने के लिए किसी की जरूरत है. मैंने उनसे स्पिनरों के खिलाफ बल्लेबाजी में कदमों के इस्तेमाल के तरीके पर बातचीत की.'
'''

In [155]:
damping = 0.85  # damping coefficient, usually is .85
min_diff = 1e-5  # convergence threshold
steps = 100  # iteration steps
sentences = None
pr_vector = None

In [156]:
def _replace_whitespace(match):
    text = match.group()

    if "\n" in text or "\r" in text:
        return "\n"
    else:
        return " "

In [157]:
def normalize_whitespace(text):
    """
    Translates multiple whitespace into single space character.
    If there is at least one new line character chunk is replaced
    by single LF (Unix new line) character.
    """
    return MULTIPLE_WHITESPACE_PATTERN.sub(_replace_whitespace, text)

In [158]:
def get_top_sentences( number=5):

        top_sentences = []

        if pr_vector is not None:

            sorted_pr = np.argsort(pr_vector)
            sorted_pr = list(sorted_pr)
            sorted_pr.reverse()

            index = 0
            for epoch in range(number):
                sent = sentences[sorted_pr[index]]
                sent = normalize_whitespace(sent)
                top_sentences.append(sent)
                index += 1

        return top_sentences

In [159]:
def _run_page_rank(similarity_matrix):

        pr_vector = np.array([1] * len(similarity_matrix))

        # Iteration
        previous_pr = 0
        for epoch in range(steps):
            pr_vector = (1 - damping) + damping * np.matmul(similarity_matrix, pr_vector)
            if abs(previous_pr - sum(pr_vector)) < min_diff:
                break
            else:
                previous_pr = sum(pr_vector)

        return pr_vector

In [160]:
def core_cosine_similarity(vector1, vector2):
    """
    measure cosine similarity between two vectors
    :param vector1:
    :param vector2:
    :return: 0 < cosine similarity value < 1
    """
    return 1 - cosine_distance(vector1, vector2)

In [161]:
def get_symmetric_matrix(matrix):
    """
    Get Symmetric matrix
    :param matrix:
    :return: matrix
    """
    return matrix + matrix.T - np.diag(matrix.diagonal())

In [162]:
def _sentence_similarity(sent1, sent2, stopwords=None):
        if stopwords is None:
            stopwords = []

        sent1 = [w.lower() for w in sent1]
        sent2 = [w.lower() for w in sent2]

        all_words = list(set(sent1 + sent2))

        vector1 = [0] * len(all_words)
        vector2 = [0] * len(all_words)

        # build the vector for the first sentence
        for w in sent1:
            if w in stopwords:
                continue
            vector1[all_words.index(w)] += 1

        # build the vector for the second sentence
        for w in sent2:
            if w in stopwords:
                continue
            vector2[all_words.index(w)] += 1

        return core_cosine_similarity(vector1, vector2)

In [163]:
def _build_similarity_matrix(sentences, stopwords=None):
    # create an empty similarity matrix
    sm = np.zeros([len(sentences), len(sentences)])

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:
                continue

            sm[idx1][idx2] = _sentence_similarity(sentences[idx1], sentences[idx2], stopwords=stopwords)

    # Get Symmeric matrix
    sm = get_symmetric_matrix(sm)

    # Normalize matrix by column
    norm = np.sum(sm, axis=0)
    sm_norm = np.divide(sm, norm, where=norm != 0)  # this is ignore the 0 element in norm

    return sm_norm

In [164]:
def analyze(text, stop_words=None):
        global sentences
        global pr_vector
        text_str = text
        sentences = sent_tokenize(text_str)
        tokenized_sentences = [word_tokenize(sent) for sent in sentences]
        # print(tokenized_sentences)
        # print(_sentence_similarity(tokenized_sentences[0],tokenized_sentences[1]))
        # _build_similarity_matrix(tokenized_sentences, stop_words)
        similarity_matrix = _build_similarity_matrix(tokenized_sentences, stop_words)
        pr_vector = _run_page_rank(similarity_matrix)

In [165]:
analyze(text_str)

In [166]:
print(get_top_sentences(5))

["मैंने उनसे स्पिनरों के खिलाफ बल्लेबाजी में कदमों के इस्तेमाल के तरीके पर बातचीत की.'", '\nभारतीय टीम को ऑस्ट्रेलिया के खिलाफ 4 मैचों की टेस्ट सीरीज का तीसरा मुकाबला 1 मार्च से इंदौर में खेलना है.', 'हैंड्सकॉम्ब ने 2016 में IPL के दौरान रहाणे के साथ ड्रेसिंग रूम शेयर किया था.', 'स्पिनरों के खिलाफ रहाणे की बल्लेबाजी देख कर मैं सोच रहा था कि मुझे यह सिखाने के लिए किसी की जरूरत है.', 'मगर मिडिल ऑर्डर बल्लेबाज पीटर हैंड्सकॉम्ब ने दिल्ली टेस्ट में 72 रनों की नाबाद पारी खेली थी.पीटर हैंड्सकॉम्ब ने पूर्व भारतीय उपकप्तान अजिंक्य रहाणे के साथ उस बातचीत को याद किया जिससे उन्हें उपमहाद्वीप में स्पिन गेंदबाजी का सामना करने में मदद की.']
